# Whole-body manipulation
The objective of this exercise is to reach multiple targets while keeping balance in the Talos humanoid robot.

<img src="https://robots.ros.org/assets/img/robots/talos/TALOS-Wave.jpg" alt="drawing" width="250"/>


This exercise focuses on a multi-contact optimal control problem of the form:

\begin{equation}\nonumber
	\begin{aligned}
		\min_{\mathbf{x}_s,\mathbf{u}_s}
		&\hspace{-2.em}
		& & \hspace{-0.75em}l_N(\mathbf{x}_{N})+\sum_{k=0}^{N-1} \int_{t_k}^{t_k+\Delta t_k}\hspace{-2.em} l_k(\mathbf{x}_k,\mathbf{u}_k)dt \hspace{-8.em}&\\
		& \hspace{-1em}\textrm{s.t.}
		& & \mathbf{q}_{k+1} = \mathbf{q}_k \oplus \int_{t_k}^{t_k+\Delta t_k}\hspace{-2.em}\mathbf{v}_{k+1}\,dt, &\textrm{(integrator)}\\
		& & & \mathbf{v}_{k+1} = \mathbf{v}_k + \int_{t_k}^{t_k+\Delta t_k}\hspace{-2.em}\mathbf{\dot{v}}_k\,dt, &\\
		& & & \hspace{-1em}\left[\begin{matrix}\mathbf{\dot{v}}_k \\ -\boldsymbol{\lambda}_k\end{matrix}\right] =
		\left[\begin{matrix}\mathbf{M} & \mathbf{J}^{\top}_c \\ {\mathbf{J}_{c}} & \mathbf{0} \end{matrix}\right]^{-1}
		\left[\begin{matrix}\boldsymbol{\tau}_b \\ -\mathbf{a}_0 \\\end{matrix}\right], &\textrm{(contact dynamics)}\\
		& & & \mathbf{R}\boldsymbol{\lambda}_{\mathcal{C}(k)} \leq \mathbf{\mathbf{r}}, &\textrm{(friction-cone)}\\
			& & & \mathbf{\bar{x}} \leq \mathbf{x}_k \leq \mathbf{\underline{x}}, &\textrm{(state bounds)}
		\end{aligned}
\end{equation}

where $l_i(\mathbf{x}_i, \mathbf{u}_i) = w_{hand}\|\log{(\mathbf{p}_{\mathcal{G}(k)}(\mathbf{q}_k)^{-1} \mathbf{^oM}_{\mathbf{f}_{\mathcal{G}(k)}})}\| + w_{xreg}\|\mathbf{x} - \mathbf{x}_0\|_{Q} + w_{ureg}\|\mathbf{u}\|_{R}$. Note that (1) the first term is the hand placement cost and (2) the terminal cost does not include the control regularization term.

Below there is a basic example that defines the above problem for reaching one target. Later, you will have to build the problem on top of it.

Without no more preamble, let's first declare the robot model and the foot and hand names!

In [4]:
import crocoddyl
from example_robot_data.robots_loader import RobotLoader
import numpy as np
import pinocchio as pin

urdf_path = "/home/pdebbad/ros2_dev_ws/talos_harmonic/src/guides/notebooks/talos_description/robots/talos.urdf"
srdf_path = "/home/pdebbad/ros2_dev_ws/talos_harmonic/src/guides/notebooks/talos_description/srdf/talos.srdf"

# Load robot
robot = pin.robot_wrapper.RobotWrapper.BuildFromURDF(
    str(urdf_path), 
    ["/home/pdebbad/ros2_dev_ws/talos_harmonic/src/guides/notebooks"],
    pin.JointModelFreeFlyer()
)
rmodel = robot.model

pin.loadRotorParameters(rmodel, srdf_path, False)
rmodel.armature = np.multiply(
    rmodel.rotorInertia.flat, np.square(rmodel.rotorGearRatio.flat)
)
pin.loadReferenceConfigurations(rmodel, srdf_path, False)
q0 = pin.neutral(rmodel)
q0 = rmodel.referenceConfigurations["half_sitting"].copy()
q0 = pin.normalize(rmodel, q0)
# for key in rmodel.referenceConfigurations:
#     print(key)
# for frame in rmodel.frames:
#     print(frame.name)
x0 = np.concatenate([q0, np.zeros(rmodel.nv)])


# Declaring the foot and hand names
rf_name = "right_sole_link"
lf_name = "left_sole_link"
lh_name = "gripper_left_joint"
rh_name = "gripper_right_joint"

# Getting the frame ids
rf_id = rmodel.getFrameId(rf_name)
lf_id = rmodel.getFrameId(lf_name)
lh_id = rmodel.getFrameId(lh_name)
rh_id = rmodel.getFrameId(rh_name)

# Define the robot's state and actuation
state = crocoddyl.StateMultibody(rmodel)
actuation = crocoddyl.ActuationModelFloatingBase(state)

In [5]:
# import crocoddyl
# import example_robot_data
# import numpy as np
# import pinocchio as pin

# # Load robot
# robot = example_robot_data.load("talos_full")
# rmodel = robot.model
# # for key in rmodel.referenceConfigurations:
# #     print(key)
# # for frame in rmodel.frames:
# #     print(frame.name)
# q0 = rmodel.referenceConfigurations["half_sitting"]
# x0 = np.concatenate([q0, np.zeros(rmodel.nv)])

# # Declaring the foot and hand names
# rf_name = "right_sole_link"
# lf_name = "left_sole_link"
# lh_name = "gripper_left_joint"

# # Getting the frame ids
# rf_id = rmodel.getFrameId(rf_name)
# lf_id = rmodel.getFrameId(lf_name)
# lh_id = rmodel.getFrameId(lh_name)

# # Define the robot's state and actuation
# state = crocoddyl.StateMultibody(rmodel)

# actuation = crocoddyl.ActuationModelFloatingBase(state)

With the following function, we could build a differential action model giving a desired hand target.
The function builds a double-support contact phase and defines a hand-placement task. The cost function also includes:
 - state and control regularization terms
 - state limits penalization
 - friction cone penalization


In [6]:
def createActionModel(target):
    # Creating a double-support contact (feet support)
    contacts = crocoddyl.ContactModelMultiple(state, actuation.nu)
    lf_contact = crocoddyl.ContactModel6D(
        state,
        lf_id,
        pin.SE3.Identity(),
        pin.LOCAL_WORLD_ALIGNED,
        actuation.nu,
        np.array([0, 0]),
    )
    rf_contact = crocoddyl.ContactModel6D(
        state,
        rf_id,
        pin.SE3.Identity(),
        pin.LOCAL_WORLD_ALIGNED,
        actuation.nu,
        np.array([0, 0]),
    )
    contacts.addContact("lf_contact", lf_contact)
    contacts.addContact("rf_contact", rf_contact)

    # Define the cost sum (cost manager)
    costs = crocoddyl.CostModelSum(state, actuation.nu)

    # Adding the hand-placement cost
    w_hand = np.array([1] * 3 + [0.0001] * 3)
    lh_Mref = pin.SE3(np.eye(3), target)
    activation_hand = crocoddyl.ActivationModelWeightedQuad(w_hand**2)
    lh_cost = crocoddyl.CostModelResidual(
        state,
        activation_hand,
        crocoddyl.ResidualModelFramePlacement(state, lh_id, lh_Mref, actuation.nu),
    )
    costs.addCost("lh_goal", lh_cost, 1e2)

    # Adding state and control regularization terms
    w_x = np.array([0] * 3 + [10.0] * 3 + [0.01] * (state.nv - 6) + [10] * state.nv)
    activation_xreg = crocoddyl.ActivationModelWeightedQuad(w_x**2)
    x_reg_cost = crocoddyl.CostModelResidual(
        state, activation_xreg, crocoddyl.ResidualModelState(state, x0, actuation.nu)
    )
    u_reg_cost = crocoddyl.CostModelResidual(
        state, crocoddyl.ResidualModelControl(state, actuation.nu)
    )
    costs.addCost("xReg", x_reg_cost, 1e-3)
    costs.addCost("uReg", u_reg_cost, 1e-4)

    # Adding the state limits penalization
    # x_lb = state.lb
    # x_ub = state.ub
    # activation_xbounds = crocoddyl.ActivationModelQuadraticBarrier(
    #     crocoddyl.ActivationBounds(x_lb, x_ub)
    # )
    # x_bounds = crocoddyl.CostModelResidual(
    #     state,
    #     activation_xbounds,
    #     crocoddyl.ResidualModelState(state, 0 * x0, actuation.nu),
    # )
    # costs.addCost("xBounds", x_bounds, 1.0)

    # Adding the friction cone penalization
    nsurf, mu = np.identity(3), 0.7
    cone = crocoddyl.FrictionCone(nsurf, mu, 4, False)
    activation_friction = crocoddyl.ActivationModelQuadraticBarrier(
        crocoddyl.ActivationBounds(cone.lb, cone.ub)
    )
    lf_friction = crocoddyl.CostModelResidual(
        state,
        activation_friction,
        crocoddyl.ResidualModelContactFrictionCone(state, lf_id, cone, actuation.nu),
    )
    rf_friction = crocoddyl.CostModelResidual(
        state,
        activation_friction,
        crocoddyl.ResidualModelContactFrictionCone(state, rf_id, cone, actuation.nu),
    )
    costs.addCost("lf_friction", lf_friction, 1e1)
    costs.addCost("rf_friction", rf_friction, 1e1)

    # Creating the action model
    dmodel = crocoddyl.DifferentialActionModelContactFwdDynamics(
        state, actuation, contacts, costs
    )
    return dmodel

And to easily build a sequence of tasks, we have the following function

In [7]:
def createSequence(dmodels, DT, N):
    return [
        [crocoddyl.IntegratedActionModelEuler(m, DT)] * N
        + [crocoddyl.IntegratedActionModelEuler(m, 0.0)]
        for m in dmodels
    ]

Finally, the following function allows us to display the motions and desired targets:

In [8]:
import meshcat.geometry as g
import meshcat.transformations as tf


def createDisplay(targets):
    display = crocoddyl.MeshcatDisplay(robot, 4, 4, False)
    for i, target in enumerate(targets):
        display.robot.viewer["target_" + str(i)].set_object(g.Sphere(0.05))
        Href = np.array(
            [
                [1.0, 0.0, 0.0, target[0]],
                [0.0, 1.0, 0.0, target[1]],
                [0.0, 0.0, 1.0, target[2]],
                [0.0, 0.0, 0.0, 1.0],
            ]
        )
        display.robot.viewer["target_" + str(i)].set_transform(
            np.array(
                [
                    [1.0, 0.0, 0.0, target[0]],
                    [0.0, 1.0, 0.0, target[1]],
                    [0.0, 0.0, 1.0, target[2]],
                    [0.0, 0.0, 0.0, 1.0],
                ]
            )
        )
    return display

Now, we create an optimal control problem to reach a single target

In [9]:
DT, N = 5e-2, 20
target = np.array([0.4, 0, 1.2])

# Creating a running model for the target
dmodel = createActionModel(target)
seqs = createSequence([dmodel], DT, N)

# Defining the problem and the solver
problem = crocoddyl.ShootingProblem(x0, sum(seqs, [])[:-1], seqs[-1][-1])
fddp = crocoddyl.SolverFDDP(problem)

# # Creating display
display = createDisplay([target])

# Adding callbacks to inspect the evolution of the solver (logs are printed in the terminal)
fddp.setCallbacks([crocoddyl.CallbackVerbose(), crocoddyl.CallbackDisplay(display)])

# Embedded in this cell
display.robot.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


Let's solve this problem!

In [10]:
print("Problem solved:", fddp.solve())
print("Number of iterations:", fddp.iter)
print("Total cost:", fddp.cost)
print("Gradient norm:", fddp.stoppingCriteria())

iter    cost       merit      stop      |grad|      preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi
   0  1.566e+02  0.000e+00  2.484e+02  4.601e+02  1.000e-09  1.000e-09  0.2500  6.099e+00  0.000e+00  0.000e+00  1.018e+02  9.809e+01  0.000e+00  0.000e+00
   1  3.426e+01  0.000e+00  1.520e+02  2.867e+02  1.000e-09  1.000e-09  0.5000  4.574e+00  0.000e+00  0.000e+00  1.097e+02  1.224e+02  0.000e+00  0.000e+00
   2  1.272e+01  0.000e+00  3.086e+01  5.665e+01  1.000e-09  1.000e-09  0.5000  2.287e+00  0.000e+00  0.000e+00  2.188e+01  2.153e+01  0.000e+00  0.000e+00
   3  5.694e+00  0.000e+00  9.690e+00  1.757e+01  1.000e-09  1.000e-09  0.5000  1.144e+00  0.000e+00  0.000e+00  6.816e+00  7.029e+00  0.000e+00  0.000e+00
   4  3.746e+00  0.000e+00  2.868e+00  5.019e+00  1.000e-09  1.000e-09  0.5000  5.718e-01  0.000e+00  0.000e+00  1.972e+00  1.948e+00  0.000e+00  0.000e+00
   5  2.914e+00  0.000e+00  1.003e+00  1.842e+00  1.000e-09  1.000e

You could display again the final solution

In [11]:
display.rate = -1
display.freq = 1
display.displayFromSolver(fddp)

## Modifying the example

Let's build an optimal control problem to reach 4 targets as described below:

In [12]:
targets = []
targets += [np.array([0.4, 0.1, 1.2])]
targets += [np.array([0.6, 0.1, 1.2])]
targets += [np.array([0.6, -0.1, 1.2])]
targets += [np.array([0.4, -0.1, 1.2])]

Now let's display the targets in Meshcat. Do not forget to embed again the display into the jupyter cell

In [13]:
display = createDisplay(targets)

# Embedded in this cell
display.robot.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


After checking that everything is alright, it's time to build the sequence!
Do not forget to create the problem as well :)

In [14]:
DT, N = 5e-2, 20

# Creating a running model for the target
dmodel0 = createActionModel(targets[0])
dmodel1 = createActionModel(targets[1])
dmodel2 = createActionModel(targets[2])
dmodel3 = createActionModel(targets[3])
seqs = createSequence([dmodel0, dmodel1, dmodel2, dmodel3], DT, N)

# Defining the problem and the solver
problem = crocoddyl.ShootingProblem(x0, sum(seqs, [])[:-1], seqs[-1][-1])
fddp = crocoddyl.SolverFDDP(problem)

And we solve it as before

In [15]:
# Create the FDDP solver
fddp = crocoddyl.SolverFDDP(problem)
fddp.setCallbacks([crocoddyl.CallbackVerbose(), crocoddyl.CallbackDisplay(display)])

# Solves the problem
print("Problem solved:", fddp.solve())
print("Number of iterations:", fddp.iter)
print("Total cost:", fddp.cost)
print("Gradient norm:", fddp.stoppingCriteria())

iter    cost       merit      stop      |grad|      preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi
   0  5.829e+02  0.000e+00  6.553e+02  1.264e+03  1.000e-09  1.000e-09  0.0625  6.099e+00  0.000e+00  0.000e+00  7.662e+01  7.935e+01  0.000e+00  0.000e+00
   1  5.121e+02  0.000e+00  5.767e+02  1.112e+03  1.000e-09  1.000e-09  0.0625  5.718e+00  0.000e+00  0.000e+00  6.744e+01  7.076e+01  0.000e+00  0.000e+00
   2  4.498e+02  0.000e+00  5.066e+02  9.793e+02  1.000e-09  1.000e-09  0.0625  5.361e+00  0.000e+00  0.000e+00  5.936e+01  6.228e+01  0.000e+00  0.000e+00
   3  3.952e+02  0.000e+00  4.448e+02  8.622e+02  1.000e-09  1.000e-09  0.0625  5.025e+00  0.000e+00  0.000e+00  5.226e+01  5.465e+01  0.000e+00  0.000e+00
   4  3.473e+02  0.000e+00  3.906e+02  7.588e+02  1.000e-09  1.000e-09  0.0625  4.711e+00  0.000e+00  0.000e+00  4.598e+01  4.789e+01  0.000e+00  0.000e+00
   5  2.730e+02  0.000e+00  3.430e+02  6.712e+02  1.000e-09  1.000e

Do not miss the change to display the motion at the right display speed!

In [16]:
display.rate = -1
display.freq = 1
display.displayFromSolver(fddp)

## Same targets with right hand

You've learned how to reach 4 targets with the left hand, congratulations!

To keep playing within this problem, you should create a new createActionModel to achieve the same task for the right hand.

In [17]:
def createActionModel(target):
    # Creating a double-support contact (feet support)
    contacts = crocoddyl.ContactModelMultiple(state, actuation.nu)
    lf_contact = crocoddyl.ContactModel6D(
        state,
        lf_id,
        pin.SE3.Identity(),
        pin.LOCAL_WORLD_ALIGNED,
        actuation.nu,
        np.array([0, 0]),
    )
    rf_contact = crocoddyl.ContactModel6D(
        state,
        rf_id,
        pin.SE3.Identity(),
        pin.LOCAL_WORLD_ALIGNED,
        actuation.nu,
        np.array([0, 0]),
    )
    contacts.addContact("lf_contact", lf_contact)
    contacts.addContact("rf_contact", rf_contact)

    # Define the cost sum (cost manager)
    costs = crocoddyl.CostModelSum(state, actuation.nu)

    # Adding the hand-placement cost
    w_hand = np.array([1] * 3 + [0.0001] * 3)
    rh_Mref = pin.SE3(np.eye(3), target)
    activation_hand = crocoddyl.ActivationModelWeightedQuad(w_hand**2)
    rh_cost = crocoddyl.CostModelResidual(
        state,
        activation_hand,
        crocoddyl.ResidualModelFramePlacement(state, rh_id, rh_Mref, actuation.nu),
    )
    costs.addCost("rh_goal", rh_cost, 1e2)

    # Adding state and control regularization terms
    w_x = np.array([0] * 3 + [10.0] * 3 + [0.01] * (state.nv - 6) + [10] * state.nv)
    activation_xreg = crocoddyl.ActivationModelWeightedQuad(w_x**2)
    x_reg_cost = crocoddyl.CostModelResidual(
        state, activation_xreg, crocoddyl.ResidualModelState(state, x0, actuation.nu)
    )
    u_reg_cost = crocoddyl.CostModelResidual(
        state, crocoddyl.ResidualModelControl(state, actuation.nu)
    )
    costs.addCost("xReg", x_reg_cost, 1e-3)
    costs.addCost("uReg", u_reg_cost, 1e-4)

    # Adding the state limits penalization
    # x_lb = state.lb
    # x_ub = state.ub
    # activation_xbounds = crocoddyl.ActivationModelQuadraticBarrier(
    #     crocoddyl.ActivationBounds(x_lb, x_ub)
    # )
    # x_bounds = crocoddyl.CostModelResidual(
    #     state,
    #     activation_xbounds,
    #     crocoddyl.ResidualModelState(state, 0 * x0, actuation.nu),
    # )
    # costs.addCost("xBounds", x_bounds, 1.0)

    # Adding the friction cone penalization
    nsurf, mu = np.identity(3), 0.7
    cone = crocoddyl.FrictionCone(nsurf, mu, 4, False)
    activation_friction = crocoddyl.ActivationModelQuadraticBarrier(
        crocoddyl.ActivationBounds(cone.lb, cone.ub)
    )
    lf_friction = crocoddyl.CostModelResidual(
        state,
        activation_friction,
        crocoddyl.ResidualModelContactFrictionCone(state, lf_id, cone, actuation.nu),
    )
    rf_friction = crocoddyl.CostModelResidual(
        state,
        activation_friction,
        crocoddyl.ResidualModelContactFrictionCone(state, rf_id, cone, actuation.nu),
    )
    costs.addCost("lf_friction", lf_friction, 1e1)
    costs.addCost("rf_friction", rf_friction, 1e1)

    # Creating the action model
    dmodel = crocoddyl.DifferentialActionModelContactFwdDynamics(
        state, actuation, contacts, costs
    )
    return dmodel

And here you need to create the problem and solve.
Do not forget to display the results

In [18]:
DT, N = 5e-2, 20

# Creating a running model for the target
dmodel0 = createActionModel(targets[0])
dmodel1 = createActionModel(targets[1])
dmodel2 = createActionModel(targets[2])
dmodel3 = createActionModel(targets[3])
seqs = createSequence([dmodel0, dmodel1, dmodel2, dmodel3], DT, N)

# Defining the problem and the solver
problem = crocoddyl.ShootingProblem(x0, sum(seqs, [])[:-1], seqs[-1][-1])
fddp = crocoddyl.SolverFDDP(problem)

In [19]:
# Create the FDDP solver
fddp = crocoddyl.SolverFDDP(problem)
fddp.setCallbacks([crocoddyl.CallbackVerbose(), crocoddyl.CallbackDisplay(display)])
x0 = problem.x0
T = problem.T
xs_init = [x0] * (T+1)

# Solves the problem
print("Problem solved:", fddp.solve())
print("Number of iterations:", fddp.iter)
print("Total cost:", fddp.cost)
print("Gradient norm:", fddp.stoppingCriteria())

iter    cost       merit      stop      |grad|      preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi
   0  5.917e+02  0.000e+00  7.679e+02  1.553e+03  1.000e-09  1.000e-09  0.1250  6.099e+00  0.000e+00  0.000e+00  1.819e+02  1.859e+02  0.000e+00  0.000e+00
   1  4.608e+02  0.000e+00  5.852e+02  1.165e+03  1.000e-09  1.000e-09  0.1250  5.337e+00  0.000e+00  0.000e+00  1.365e+02  1.309e+02  0.000e+00  0.000e+00
   2  3.522e+02  0.000e+00  4.545e+02  8.982e+02  1.000e-09  1.000e-09  0.1250  4.670e+00  0.000e+00  0.000e+00  1.053e+02  1.086e+02  0.000e+00  0.000e+00
   3  2.696e+02  0.000e+00  3.459e+02  6.800e+02  1.000e-09  1.000e-09  0.1250  4.086e+00  0.000e+00  0.000e+00  7.978e+01  8.257e+01  0.000e+00  0.000e+00
   4  2.070e+02  0.000e+00  2.634e+02  5.155e+02  1.000e-09  1.000e-09  0.1250  3.575e+00  0.000e+00  0.000e+00  6.050e+01  6.266e+01  0.000e+00  0.000e+00
   5  1.596e+02  0.000e+00  2.008e+02  3.921e+02  1.000e-09  1.000e

In [20]:
display.rate = -1
display.freq = 1
display.displayFromSolver(fddp)

In [25]:
print(fddp.K[0].shape)

(44, 100)


In [26]:
print(len(fddp.K))

83


In [27]:
print(robot.nq)

51
